In [ ]:
import numpy as np
from sklearn import metrics

def conv2Log(dFrame, fldList):
    for item in fldList:
        dFrame[item] = np.log(dFrame[item])

def rmsle(y_true, y_pred):
    assert len(y_true) == len(y_pred)
    return np.square(np.log(y_pred + 1) - np.log(y_true + 1)).mean() ** 0.5

dlist = ['MSSubClass','MSZoning','LotShape','LandContour','LotConfig','LandSlope','Neighborhood','Condition1','Condition2',\
         'BldgType','HouseStyle','Overall_Composite','Exterior1st','Exterior2nd','MasVnrType','Exter_Composite',\
         'Bsmt_Composite','BsmtExposure','BsmtFinType1','BsmtFinType2','HeatingQC','Electrical','KitchenQual','Functional',\
         'GarageType','GarageFinish','GarageCars','Garage_Composite','SaleType','SaleCondition']


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing

%matplotlib inline
#panda is a read/write tool

train_file = "c:\\house\\train.csv"
test_file = "c:\\house\\test.csv"

# comma delimited is the default
hsData = pd.read_csv(train_file, header = 0)
# below fileds are drop due to many empty data
hsData = hsData.drop(['MiscFeature','MiscVal','Alley'], axis=1)
# blew field are drop due to 300- empty data, need consider
hsData = hsData.drop(['LotFrontage'], axis=1)
# blew field are drop due to crazy skew
hsData = hsData.drop(['Utilities','YrSold','MoSold'], axis=1)
# below fileds are drop due to bad negative correlation
hsData = hsData.drop(['KitchenAbvGr','EnclosedPorch','LowQualFinSF'], axis=1)
# blew field is drop due to 800+ zeros
hsData = hsData.drop(['MasVnrArea','BsmtFinSF2'], axis=1)


hsData['MSSubClass'] = hsData['MSSubClass'].apply(str)
hsData['MSSubClass'].replace(['180','30','45','190','50','90','85','40','160','70','20','75','80','120','60'], [88500,99900,107500,128250,132000,135980,140750,142500,146000,156000,159250,163500,166500,192000,215200], inplace=True)

hsData['MSZoning'].replace(['C (all)','RM','RH','RL','FV'], [74700,120500,136500,174000,205950], inplace=True)

hsData['LotShape'].replace(['Reg','IR1','IR3','IR2'], [146000,189000,203570,221000], inplace=True)
hsData['LandContour'].replace(['Bnk','Lvl','Low','HLS'], [139400,162900,190000,222250], inplace=True)
hsData['LotConfig'].replace(['Inside','Corner','FR2','FR3','CulDSac'], [159697,160000,165000,195450,199262], inplace=True)

hsData['LandSlope'].replace(['Gtl','Sev','Mod'], [161875,185000,186700], inplace=True)

hsData['Neighborhood'].replace(['MeadowV','IDOTRR','BrDale','OldTown','Edwards','BrkSide','Sawyer','Blueste','SWISU','NAmes','NPkVill','Mitchel','SawyerW','Gilbert','NWAmes','Blmngtn','CollgCr','ClearCr','Crawfor','Veenker','Somerst','Timber','StoneBr','NoRidge','NridgHt'], \
                                [88000,103000,106000,119000,121750,124300,135000,137500,139500,140000,146000,153500,179900,181000,182900,191000,197200,200250,200624,218000,225500,228475,278000,301500,315000], inplace=True)

hsData['Condition1'].replace(['Artery','Feedr','RRAe','Norm','RRAn','RRNe','PosN','PosA','RRNn'], [119550,140000,142500,166500,171495,190750,200000,212500,214000], inplace=True)
hsData['Condition2'].replace(['RRNn','Artery','Feedr','RRAn','Norm','RRAe','PosN','PosA'], [96750,106500,127500,136905,163500,190000,284875,325000], inplace=True)

hsData['BldgType'].replace(['2fmCon','Duplex','Twnhs','1Fam','TwnhsE'], [127500,135980,137500,167900,172200], inplace=True)

hsData['HouseStyle'].replace(['1.5Unf','1.5Fin','2.5Unf','SFoyer','1Story','SLvl','2Story','2.5Fin'], [111250,132000,133900,135960,154750,164500,190000,194000], inplace=True)

#hsData['OverallQual'].replace([1,2,3,4,5,6,7,8,9,10], [50150,60000,86250,108000,133000,160000,200141,269750,345000,432390], inplace=True)
#hsData['OverallCond'].replace([1,2,3,4,8,6,7,9,5], [61000,85000,89500,115000,142500,142750,145000,176200,185000], inplace=True)
#print hsData['OverallCond'].unique()

hsData['Overall_Composite'] = hsData['OverallQual'] + hsData['OverallCond']
hsData['Overall_Composite'].replace([3,4,2,5,6,7,8,9,10,11,12,13,14,16,17,15,19], [39300,39300,61000,68250,74500,92900,93000,115000,129500,146900,179470,230000,247200,250250,299800,310000,400000], inplace=True)
# below fileds are drop due to composite data
hsData = hsData.drop(['OverallQual','OverallCond'], axis=1)

#hsData['Year_Composite'] = hsData['YearBuilt'] + hsData['YearRemodAdd']
#hsData['Year_Composite'] = hsData['Year_Composite'].apply(lambda year: year - 3700)
# YearBuilt and YearRemodAdd will drop shortly

hsData['Exterior1st'].replace(['BrkComm','AsphShn','CBlock','AsbShng','WdShing','Wd Sdng','MetalSd','Stucco','HdBoard','BrkFace','Plywood','VinylSd','CemntBd','Stone','ImStucc'], [71000,100000,105000,108000,128700,138943,139000,144000,149900,165750,167450,200000,236500,258500,262000], inplace=True)
hsData['Exterior2nd'].replace(['CBlock','AsbShng','Wd Sdng','Wd Shng','MetalSd','AsphShn','Stucco','Brk Cmn','HdBoard','BrkFace','Plywood','Stone','ImStucc','VinylSd','CmentBd','Other'], [105000,111000,138000,138225,138750,139000,142000,147000,155000,160000,160750,177000,187600,200070,238750,319000], inplace=True)

hsData['MasVnrType'] = hsData['MasVnrType'].fillna('None')
hsData['MasVnrType'].replace(['BrkCmn','None','BrkFace','Stone'], [139000,143000,181000,246839], inplace=True)

hsData['Exter_Composite'] = hsData['ExterQual'] + hsData['ExterCond']
hsData['Exter_Composite'].replace(['FaFa','TAPo','FaTA','TAFa','TAEx','TATA','TAGd','GdGd','GdTA','ExGd','ExEx','ExTA'], \
                             [60500,76500,99000,118732,139500,139950,140000,210000,221000,266500,325000,374189], inplace=True)

hsData['BsmtQual'] = hsData['BsmtQual'].fillna('NA')
hsData['BsmtCond'] = hsData['BsmtCond'].fillna('NA')
hsData['Bsmt_Composite'] = hsData['BsmtQual'] + hsData['BsmtCond']
hsData['Bsmt_Composite'].replace(['FaPo','NANA','FaTA','FaFa','FaGd','TAPo','TAFa','TATA','TAGd','GdFa','GdTA','GdGd','ExPo','ExGd','ExTA'], \
                             [64000,101800,112000,114875,114875,117000,117000,135625,164500,182500,192000,199000,199000,274725,318030], inplace=True)

hsData['BsmtExposure'] = hsData['BsmtExposure'].fillna('NA')
hsData['BsmtExposure'].replace(['NA','No','Mn','Av','Gd'], [104025,154000,182450,185850,226975], inplace=True)

hsData['BsmtFinType1'] = hsData['BsmtFinType1'].fillna('NA')
hsData['BsmtFinType1'].replace(['NA','LwQ','BLQ','Rec','ALQ','Unf','GLQ'], [101800,139000,139100,142000,149250,161750,213750], inplace=True)

hsData['BsmtFinType2'] = hsData['BsmtFinType2'].fillna('NA')
hsData['BsmtFinType2'].replace(['NA','BLQ','Rec','LwQ','Unf','ALQ','GLQ'], [104025,143000,148750,154000,167000,174900,203125], inplace=True)

# The composite 'Total_Bath' improves a bit
hsData['Total_Bath'] = hsData['FullBath'] + hsData['BsmtFullBath'] + hsData['HalfBath']*0.5 + hsData['BsmtHalfBath']*0.5

hsData['HeatingQC'].replace(['Po','Fa','TA','Gd','Ex'], [87000,123500,135000,152000,194700], inplace=True)

hsData['CentralAir'] = hsData['CentralAir'].apply(lambda air: 1 if air == 'Y' else 0)

hsData['Electrical'] = hsData['Electrical'].fillna('SBrkr')
hsData['Electrical'].replace(['Mix','FuseP','FuseF','FuseA','SBrkr'], [67000,82000,115000,121250,170000], inplace=True)

# The composite 'Total_FlrSF' improves some
hsData['Total_FlrSF'] = hsData['1stFlrSF'] + hsData['2ndFlrSF']
hsData = hsData.drop(['1stFlrSF','2ndFlrSF'], axis=1)

hsData['KitchenQual'].replace(['Fa','TA','Gd','Ex'], [115000,137000,201400,316750], inplace=True)

hsData['Functional'].replace(['Maj2','Sev','Mod','Min1','Min2','Maj1','Typ'], [85000,129000,137900,139000,140000,140750,165500], inplace=True)

hsData['GarageType'] = hsData['GarageType'].fillna('NA')
hsData['GarageType'].replace(['NA','CarPort','Detchd','Basment','2Types','Attchd','BuiltIn'], [100000,108000,129500,148000,159000,185000,227500], inplace=True)

hsData['GarageFinish'] = hsData['GarageFinish'].fillna('NA')
hsData['GarageFinish'].replace(['NA','Unf','RFn','Fin'], [100000,135000,190000,215000], inplace=True)

hsData['GarageCars'].replace([0,1,2,4,5,3], [100000,128000,177750,200000,200000,295000], inplace=True)

hsData['GarageQual'] = hsData['GarageQual'].fillna('NA')
hsData['GarageCond'] = hsData['GarageCond'].fillna('NA')
hsData['Garage_Composite'] = hsData['GarageQual'] + hsData['GarageCond']
hsData['Garage_Composite'].replace(['PoPo','NANA','FaFa','FaPo','FaTA','ExEx','TAFa','TAGd','TATA','GdTA','GdGd','ExTA'], \
                             [96500,100000,109750,114500,117250,124000,125000,144000,171000,209115,227485,475000], inplace=True)

hsData['SaleType'].replace(['Oth','ConLI','COD','ConLD','ConLw','WD','CWD','New','Con'], [116050,125000,139000,140000,144000,158000,188750,247453,269600], inplace=True)
hsData['SaleCondition'].replace(['AdjLand','Abnorml','Family','Alloca','Normal','Partial'], [104000,130000,140500,148145,160000,244600], inplace=True)

#hsData['Has_Fence'] = hsData['Fence'].apply(lambda x: 0 if pd.isnull(x) else 1)
#print hsData['Has_Fence']
hsData['Has_Fireplaces'] = hsData['Fireplaces'].apply(lambda x: 1 if x > 0 else 0)
hsData['Has_Pool'] = hsData['PoolArea'].apply(lambda x: 1 if x > 0 else 0)
hsData = hsData.drop(['PoolQC','PoolArea','Fireplaces','Fence'], axis=1)
############################

# Trim below noise improves a bit

hsData = hsData[hsData["GrLivArea"] < 4000]
hsData["GrLivArea"] = np.log(hsData["GrLivArea"])

hsData = hsData[hsData["GarageArea"] < 1200]
#hsData["GarageArea"] = np.log(hsData["GarageArea"])

hsData = hsData[hsData["Total_FlrSF"] < 3300]
hsData = hsData[hsData["Total_FlrSF"] > 400]
#hsData = hsData[hsData["YearBuilt"] > 1888]

# below fileds are drop due to composite data
#hsData = hsData.drop(['YearBuilt','YearRemodAdd'], axis=1)
hsData = hsData.drop(['FullBath','BsmtFullBath','HalfBath','BsmtHalfBath'], axis=1)
# blew field is drop because GarageYrBlt==YearBuilt and GarageArea==GarageCars
hsData = hsData.drop(['GarageYrBlt','GarageArea'], axis=1)

hsData.info()
#hsData.describe()

In [ ]:
num_train_features = hsData.select_dtypes(include=[np.number])

num_train_features = num_train_features.drop(['Id'], axis=1)

corr = num_train_features.corr()
print (corr['SalePrice'].sort_values(ascending=False)[:70], '\n')

# Drop SalePrice' after evaluation
num_train_features = num_train_features.drop(['SalePrice'], axis=1)

#num_train_features.info()

conv2Log(num_train_features, dlist)

num_train_features.isnull().values.any()

null_columns=num_train_features.columns[num_train_features.isnull().any()]
num_train_features[null_columns].isnull().sum()

In [ ]:
hsTest = pd.read_csv(test_file, header = 0)
# below fileds are drop due to many empty data
hsTest = hsTest.drop(['MiscFeature','MiscVal','Alley'], axis=1)
# blew field are drop due to some empty data, need consider
hsTest = hsTest.drop(['LotFrontage'], axis=1)
# blew field are drop due to crazy skew
hsTest = hsTest.drop(['Utilities','YrSold','MoSold'], axis=1)
# below fileds are drop due to bad negative correlation
hsTest = hsTest.drop(['KitchenAbvGr','EnclosedPorch','LowQualFinSF'], axis=1)
# blew field is drop due to 800+ zeros
hsTest = hsTest.drop(['MasVnrArea','BsmtFinSF2'], axis=1)
# blew field is drop because GarageYrBlt==YearBuilt and GarageArea==GarageCars
hsTest = hsTest.drop(['GarageYrBlt','GarageArea'], axis=1)

hsTest['MSSubClass'] = hsTest['MSSubClass'].apply(str)
hsTest['MSSubClass'].replace(['180','30','45','190','50','90','85','40','160','70','20','75','80','120','150','60'], [88500,99900,107500,128250,132000,135980,140750,142500,146000,156000,159250,163500,166500,192000,192000,215200], inplace=True)

hsTest['MSZoning'] = hsTest['MSZoning'].fillna('RL')
hsTest['MSZoning'].replace(['C (all)','RM','RH','RL','FV'], [74700,120500,136500,174000,205950], inplace=True)

hsTest['LotShape'].replace(['Reg','IR1','IR3','IR2'], [146000,189000,203570,221000], inplace=True)
hsTest['LandContour'].replace(['Bnk','Lvl','Low','HLS'], [139400,162900,190000,222250], inplace=True)
hsTest['LotConfig'].replace(['Inside','Corner','FR2','FR3','CulDSac'], [159697,160000,165000,195450,199262], inplace=True)
hsTest['LandSlope'].replace(['Gtl','Sev','Mod'], [161875,185000,186700], inplace=True)

hsTest['Neighborhood'].replace(['MeadowV','IDOTRR','BrDale','OldTown','Edwards','BrkSide','Sawyer','Blueste','SWISU','NAmes','NPkVill','Mitchel','SawyerW','Gilbert','NWAmes','Blmngtn','CollgCr','ClearCr','Crawfor','Veenker','Somerst','Timber','StoneBr','NoRidge','NridgHt'], \
                                [88000,103000,106000,119000,121750,124300,135000,137500,139500,140000,146000,153500,179900,181000,182900,191000,197200,200250,200624,218000,225500,228475,278000,301500,315000], inplace=True)

hsTest['Condition1'].replace(['Artery','Feedr','RRAe','Norm','RRAn','RRNe','PosN','PosA','RRNn'], [119550,140000,142500,166500,171495,190750,200000,212500,214000], inplace=True)
hsTest['Condition2'].replace(['RRNn','Artery','Feedr','RRAn','Norm','RRAe','PosN','PosA'], [96750,106500,127500,136905,163500,190000,284875,325000], inplace=True)

hsTest['BldgType'].replace(['2fmCon','Duplex','Twnhs','1Fam','TwnhsE'], [127500,135980,137500,167900,172200], inplace=True)

hsTest['HouseStyle'].replace(['1.5Unf','1.5Fin','2.5Unf','SFoyer','1Story','SLvl','2Story','2.5Fin'], [111250,132000,133900,135960,154750,164500,190000,194000], inplace=True)

#hsTest['OverallQual'].replace([1,2,3,4,5,6,7,8,9,10], [50150,60000,86250,108000,133000,160000,200141,269750,345000,432390], inplace=True)
#hsTest['OverallCond'].replace([1,2,3,4,8,6,7,9,5], [61000,85000,89500,115000,142500,142750,145000,176200,185000], inplace=True)
#print hsTest['OverallCond'].unique()

hsTest['Overall_Composite'] = hsTest['OverallQual'] + hsTest['OverallCond']
hsTest['Overall_Composite'].replace([3,4,2,5,6,7,8,9,10,11,12,13,14,16,17,15,19], [39300,39300,61000,68250,74500,92900,93000,115000,129500,146900,179470,230000,247200,250250,299800,310000,400000], inplace=True)
# below fileds are drop due to composite data
hsTest = hsTest.drop(['OverallQual','OverallCond'], axis=1)

#hsTest['Year_Composite'] = hsTest['YearBuilt'] + hsTest['YearRemodAdd']
#hsTest['Year_Composite'] = hsTest['Year_Composite'].apply(lambda year: year - 3700)
# below fileds are drop due to composite data
#hsTest = hsTest.drop(['YearBuilt','YearRemodAdd'], axis=1)

hsTest['Exterior1st'] = hsTest['Exterior1st'].fillna('VinylSd')
hsTest['Exterior1st'].replace(['BrkComm','AsphShn','CBlock','AsbShng','WdShing','Wd Sdng','MetalSd','Stucco','HdBoard','BrkFace','Plywood','VinylSd','CemntBd','Stone','ImStucc'], [71000,100000,105000,108000,128700,138943,139000,144000,149900,165750,167450,200000,236500,258500,262000], inplace=True)
hsTest['Exterior2nd'] = hsTest['Exterior2nd'].fillna('VinylSd')
hsTest['Exterior2nd'].replace(['CBlock','AsbShng','Wd Sdng','Wd Shng','MetalSd','AsphShn','Stucco','Brk Cmn','HdBoard','BrkFace','Plywood','Stone','ImStucc','VinylSd','CmentBd','Other'], [105000,111000,138000,138225,138750,139000,142000,147000,155000,160000,160750,177000,187600,200070,238750,319000], inplace=True)

hsTest['MasVnrType'] = hsTest['MasVnrType'].fillna('None')
hsTest['MasVnrType'].replace(['BrkCmn','None','BrkFace','Stone'], [139000,143000,181000,246839], inplace=True)

hsTest['ExterQual'] = hsTest['ExterQual'].fillna('NA')
hsTest['ExterCond'] = hsTest['ExterCond'].fillna('NA')

hsTest['Exter_Composite'] = hsTest['ExterQual'] + hsTest['ExterCond']
hsTest['Exter_Composite'].replace(['FaPo','FaFa','TAPo','FaGd','FaTA','TAFa','TAEx','TATA','TAGd','GdFa','GdGd','GdTA','GdEx','ExGd','ExEx','ExTA'], \
                             [60500,60500,76500,99000,99000,118732,139500,139950,140000,210000,210000,221000,221000,266500,325000,374189], inplace=True)

hsTest['BsmtQual'] = hsTest['BsmtQual'].fillna('NA')
hsTest['BsmtCond'] = hsTest['BsmtCond'].fillna('NA')

for idx in range(0,1459):
    if hsTest['BsmtQual'].iloc[idx] == 'NA':
        if hsTest['BsmtCond'].iloc[idx] != 'NA':
            hsTest['BsmtCond'].iloc[idx] = 'NA'
        if hsTest['BsmtExposure'].iloc[idx] != 'NA':
            hsTest['BsmtExposure'].iloc[idx] = 'NA'            
        if hsTest['BsmtFinType1'].iloc[idx] != 'NA':
            hsTest['BsmtFinType1'].iloc[idx] = 'NA' 
        if hsTest['BsmtFinType2'].iloc[idx] != 'NA':
            hsTest['BsmtFinType2'].iloc[idx] = 'NA'             
        if hsTest['BsmtFinSF1'].iloc[idx] != 0:
            hsTest['BsmtFinSF1'].iloc[idx] = 0 
        #if hsTest['BsmtFinSF2'].iloc[idx] != 0:
        #    hsTest['BsmtFinSF2'].iloc[idx] = 0             
        if hsTest['BsmtUnfSF'].iloc[idx] != 0:
            hsTest['BsmtUnfSF'].iloc[idx] = 0 
        if hsTest['TotalBsmtSF'].iloc[idx] != 0:
            hsTest['TotalBsmtSF'].iloc[idx] = 0 
            
for idx in range(0,1459):
    if hsTest['BsmtCond'].iloc[idx] == 'NA':
        if hsTest['BsmtQual'].iloc[idx] != 'NA':
            #it's TA, not NA because NA could be a typo of TA
            hsTest['BsmtCond'].iloc[idx] = 'TA'
    #if hsTest['GarageYrBlt'].iloc[idx] > 2200:
    #    hsTest['GarageYrBlt'].iloc[idx] = hsTest['GarageYrBlt'].iloc[idx] - 200            
        
hsTest['Bsmt_Composite'] = hsTest['BsmtQual'] + hsTest['BsmtCond']
hsTest['Bsmt_Composite'].replace(['FaPo','NANA','FaTA','FaFa','FaGd','TAPo','TAFa','TATA','TAGd','GdFa','GdTA','GdGd','ExPo','ExGd','ExTA'], \
                             [64000,101800,112000,114875,114875,117000,117000,135625,164500,182500,192000,199000,199000,274725,318030], inplace=True)

hsTest['BsmtExposure'] = hsTest['BsmtExposure'].fillna('NA')
hsTest['BsmtExposure'].replace(['NA','No','Mn','Av','Gd'], [104025,154000,182450,185850,226975], inplace=True)

hsTest['BsmtFinType1'] = hsTest['BsmtFinType1'].fillna('NA')
hsTest['BsmtFinType1'].replace(['NA','LwQ','BLQ','Rec','ALQ','Unf','GLQ'], [101800,139000,139100,142000,149250,161750,213750], inplace=True)

hsTest['BsmtFinType2'] = hsTest['BsmtFinType2'].fillna('NA')
hsTest['BsmtFinType2'].replace(['NA','BLQ','Rec','LwQ','Unf','ALQ','GLQ'], [104025,143000,148750,154000,167000,174900,203125], inplace=True)

hsTest['FullBath'] = hsTest['FullBath'].fillna(0)
hsTest['BsmtFullBath'] = hsTest['BsmtFullBath'].fillna(0)
hsTest['HalfBath'] = hsTest['HalfBath'].fillna(0)
hsTest['BsmtHalfBath'] = hsTest['BsmtHalfBath'].fillna(0)
hsTest['Total_Bath'] = hsTest['FullBath'] + hsTest['BsmtFullBath'] + hsTest['HalfBath']*0.5 + hsTest['BsmtHalfBath']*0.5
# below fileds are drop due to composite data
hsTest = hsTest.drop(['FullBath','BsmtFullBath','HalfBath','BsmtHalfBath'], axis=1)

hsTest['HeatingQC'].replace(['Po','Fa','TA','Gd','Ex'], [87000,123500,135000,152000,194700], inplace=True)

hsTest['CentralAir'] = hsTest['CentralAir'].apply(lambda air: 1 if air == 'Y' else 0)

hsTest['Electrical'].replace(['Mix','FuseP','FuseF','FuseA','SBrkr'], [67000,82000,115000,121250,170000], inplace=True)

hsTest['Total_FlrSF'] = hsTest['1stFlrSF'] + hsTest['2ndFlrSF']
hsTest = hsTest.drop(['1stFlrSF','2ndFlrSF'], axis=1)

hsTest['KitchenQual'] = hsTest['KitchenQual'].fillna('TA')
hsTest['KitchenQual'].replace(['Po','Fa','TA','Gd','Ex'], [105565,115000,137000,201400,316750], inplace=True)

hsTest['Functional'] = hsTest['Functional'].fillna('Typ')
hsTest['Functional'].replace(['Maj2','Sev','Mod','Min1','Min2','Maj1','Typ'], [85000,129000,137900,139000,140000,140750,165500], inplace=True)

hsTest['GarageType'] = hsTest['GarageType'].fillna('NA')
hsTest['GarageType'].replace(['NA','CarPort','Detchd','Basment','2Types','Attchd','BuiltIn'], [100000,108000,129500,148000,159000,185000,227500], inplace=True)

hsTest['GarageFinish'] = hsTest['GarageFinish'].fillna('NA')
hsTest['GarageFinish'].replace(['NA','Unf','RFn','Fin'], [100000,135000,190000,215000], inplace=True)

hsTest['GarageCars'] = hsTest['GarageCars'].fillna(0)
hsTest['GarageCars'].replace([0,1,2,4,5,3], [100000,128000,177750,200000,200000,295000], inplace=True)

hsTest['GarageQual'] = hsTest['GarageQual'].fillna('NA')
hsTest['GarageCond'] = hsTest['GarageCond'].fillna('NA')
hsTest['Garage_Composite'] = hsTest['GarageQual'] + hsTest['GarageCond']
hsTest['Garage_Composite'].replace(['PoPo','PoFa','NANA','FaFa','FaPo','FaTA','ExEx','TAFa','TAPo','TAGd','TATA','GdTA','GdGd','GdEx','ExTA'], \
                             [96500,96500,100000,109750,114500,117250,124000,125000,125000,144000,171000,209115,227485,227485,475000], inplace=True)

hsTest['SaleType'] = hsTest['SaleType'].fillna('WD')
hsTest['SaleType'].replace(['Oth','ConLI','COD','ConLD','ConLw','WD','CWD','New','Con'], [116050,125000,139000,140000,144000,158000,188750,247453,269600], inplace=True)

hsTest['SaleCondition'].replace(['AdjLand','Abnorml','Family','Alloca','Normal','Partial'], [104000,130000,140500,148145,160000,244600], inplace=True)

#hsTest['Has_Fence'] = hsTest['Fence'].apply(lambda x: 0 if pd.isnull(x) else 1)
#print hsTest['Has_Fence'].unique()
hsTest['Has_Fireplaces'] = hsTest['Fireplaces'].apply(lambda x: 1 if x>0 else 0)
hsTest['Has_Pool'] = hsTest['PoolArea'].apply(lambda x: 1 if x > 0 else 0)
hsTest = hsTest.drop(['PoolQC','PoolArea','Fireplaces','Fence'], axis=1)

hsTest["GrLivArea"] = np.log(hsTest["GrLivArea"])

#hsTest['GarageArea'] = hsTest['GarageArea'].fillna(0)
#hsTest["GarageArea"] = np.log(hsTest["GarageArea"])

#hsTest.info()
#hsTest.describe()

In [ ]:
num_test_features = hsTest.select_dtypes(include=[np.number])

num_test_features = num_test_features.drop(['Id'], axis=1)

num_test_features.info()
conv2Log(num_test_features, dlist)

num_test_features.isnull().values.any()

null_columns=num_test_features.columns[num_test_features.isnull().any()]
num_test_features[null_columns].isnull().sum()

In [ ]:
# LassoCV Regressor
from sklearn.linear_model import LassoCV
from sklearn.cross_validation import train_test_split
from sklearn.externals import joblib

X = num_train_features
y = np.log(hsData['SalePrice'])

X_train, X_test, y_train, y_test = train_test_split(X, y)

lscv = LassoCV(eps=10**-7, n_alphas=75)
lscv.fit(X_train, y_train)

X_test_predict = lscv.predict(X_test)

X_test_predict = np.exp(X_test_predict)
y_test = np.exp(y_test)

print(lscv)
print("RMSLE: %s\n" % rmsle(y_test, X_test_predict))

Hs_Test = num_test_features
Hs_test_predict = lscv.predict(Hs_Test)
Hs_test_predict = np.exp(Hs_test_predict)

plt.grid(True)
plt.scatter(X_test_predict, y_test, alpha=.25,color='b') #alpha helps to show overlapping data
plt.plot([50000,500000],[50000,500000],'r--')
plt.xlabel('Predicted Price')
plt.ylabel('Actual Price')
plt.title('LassoCV Regressor')
plt.show()

# compare with the almost data
almost_perfect_file = "c:\\house\\almost_perfect.csv"
almostPref = pd.read_csv(almost_perfect_file, header = 0)
alpSlp = almostPref['SalePrice']

kg = rmsle(alpSlp, Hs_test_predict) - 0.0109768754926667
print("Kaggle Possible: %s\n" % kg)

submit_file = 'c:\\house\\submission_lscv_' + str(int(kg/0.000001)) + '.csv'
print submit_file
submission = pd.DataFrame({
        "Id": hsTest["Id"],
        "SalePrice": Hs_test_predict
    })

submission.to_csv(submit_file, index=False)

model_file = 'c:\\house\\lscv_' + str(int(kg/0.000001)) + '.pkl'
print model_file
joblib.dump(lscv, model_file) 

In [ ]:
# XG boost, by 'conda install py-xgboost'
from xgboost import XGBRegressor
from sklearn.cross_validation import train_test_split
from sklearn.externals import joblib

X = num_train_features
y = np.log(hsData['SalePrice'])

X_train, X_test, y_train, y_test = train_test_split(X, y)


xg = XGBRegressor(colsample_bytree=0.2,
                 gamma=0.0,
                 learning_rate=0.01,
                 max_depth=4,
                 min_child_weight=1.5,
                 n_estimators=7200,                                                                  
                 reg_alpha=0.9,
                 reg_lambda=0.6,
                 subsample=0.2,
                 seed=42,
                 silent=1)
'''
xg = XGBRegressor(learning_rate=0.1,
                 max_depth=5,
                 n_estimators=5000,                                                                 
                 reg_alpha=0.9,
                 reg_lambda=0.7,
                 seed=20)
'''                 
xg.fit(X_train, y_train)

X_test_predict = xg.predict(X_test)

X_test_predict = np.exp(X_test_predict)
y_test = np.exp(y_test)

print(xg)
print("RMSLE: %s\n" % rmsle(y_test, X_test_predict))

Hs_Test = num_test_features
Hs_test_predict = xg.predict(Hs_Test)
Hs_test_predict = np.exp(Hs_test_predict)

plt.grid(True)
plt.scatter(X_test_predict, y_test, alpha=.25,color='b') #alpha helps to show overlapping data
plt.plot([50000,500000],[50000,500000],'r--')
plt.xlabel('Predicted Price')
plt.ylabel('Actual Price')
plt.title('XG Boost')
plt.show()

# compare with the almost data
almost_perfect_file = "c:\\house\\almost_perfect.csv"
almostPref = pd.read_csv(almost_perfect_file, header = 0)
alpSlp = almostPref['SalePrice']

kg = rmsle(alpSlp, Hs_test_predict) - 0.0109768754926667
print("Kaggle Possible: %s\n" % kg)

submit_file = 'c:\\house\\submission_xg_' + str(int(kg/0.000001)) + '.csv'
print submit_file
submission = pd.DataFrame({
        "Id": hsTest["Id"],
        "SalePrice": Hs_test_predict
    })

submission.to_csv(submit_file, index=False)

model_file = 'c:\\house\\xg_' + str(int(kg/0.000001)) + '.pkl'
print model_file
joblib.dump(xg, model_file) 

In [ ]:
# Lasso Regressor
from sklearn.linear_model import Lasso
from sklearn.cross_validation import train_test_split
from sklearn.externals import joblib

X = num_train_features
y = np.log(hsData['SalePrice'])

X_train, X_test, y_train, y_test = train_test_split(X, y)

ls = Lasso(alpha=0.00055)
ls.fit(X_train, y_train)

X_test_predict = ls.predict(X_test)

X_test_predict = np.exp(X_test_predict)
y_test = np.exp(y_test)

print(ls)
print("RMSLE: %s\n" % rmsle(y_test, X_test_predict))

Hs_Test = num_test_features
Hs_test_predict = ls.predict(Hs_Test)
Hs_test_predict = np.exp(Hs_test_predict)

plt.grid(True)
plt.scatter(X_test_predict, y_test, alpha=.25,color='b') #alpha helps to show overlapping data
plt.plot([50000,500000],[50000,500000],'r--')
plt.xlabel('Predicted Price')
plt.ylabel('Actual Price')
plt.title('Lasso Regressor')
plt.show()

# compare with the almost data
almost_perfect_file = "c:\\house\\almost_perfect.csv"
almostPref = pd.read_csv(almost_perfect_file, header = 0)
alpSlp = almostPref['SalePrice']

kg = rmsle(alpSlp, Hs_test_predict) - 0.0109768754926667
print("Kaggle Possible: %s\n" % kg)

submit_file = 'c:\\house\\submission_ls_' + str(int(kg/0.000001)) + '.csv'
print submit_file
submission = pd.DataFrame({
        "Id": hsTest["Id"],
        "SalePrice": Hs_test_predict
    })

submission.to_csv(submit_file, index=False)

model_file = 'c:\\house\\ls_' + str(int(kg/0.000001)) + '.pkl'
print model_file
joblib.dump(ls, model_file) 

In [ ]:
# Bagging Regressor
from sklearn.ensemble import BaggingRegressor
from sklearn.cross_validation import train_test_split
from sklearn.externals import joblib

X = num_train_features
y = np.log(hsData['SalePrice'])

X_train, X_test, y_train, y_test = train_test_split(X, y)

bg = BaggingRegressor(n_estimators=15)
bg.fit(X_train, y_train)

X_test_predict = bg.predict(X_test)

X_test_predict = np.exp(X_test_predict)
y_test = np.exp(y_test)

print(bg)
print("RMSLE: %s\n" % rmsle(y_test, X_test_predict))

plt.grid(True)
plt.scatter(X_test_predict, y_test, alpha=.25,color='b') #alpha helps to show overlapping data
plt.plot([50000,500000],[50000,500000],'r--')
plt.xlabel('Predicted Price')
plt.ylabel('Actual Price')
plt.title('Bagging Regressor')
plt.show()

In [ ]:
# Random Forest Regressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.cross_validation import train_test_split
from sklearn.externals import joblib

X = num_train_features
y = np.log(hsData['SalePrice'])

X_train, X_test, y_train, y_test = train_test_split(X, y)

rf = RandomForestRegressor(n_estimators = 150, max_depth=5)
rf.fit(X_train, y_train)

X_test_predict = rf.predict(X_test)

X_test_predict = np.exp(X_test_predict)
y_test = np.exp(y_test)

print(rf)
print("RMSLE: %s\n" % rmsle(y_test, X_test_predict))

plt.grid(True)
plt.scatter(X_test_predict, y_test, alpha=.25,color='b') #alpha helps to show overlapping data
plt.plot([50000,500000],[50000,500000],'r--')
plt.xlabel('Predicted Price')
plt.ylabel('Actual Price')
plt.title('Random Forest Regressor')
plt.show()

In [ ]:
from sklearn.externals import joblib

loadLs = joblib.load('c:\\house\\ls_118820.pkl') 

Hs_Test = num_test_features
Hs_test_predict = loadLs.predict(Hs_Test)
Hs_test_predict = np.exp(Hs_test_predict)

# compare with the almost data
almost_perfect_file = "c:\\house\\almost_perfect.csv"
almostPref = pd.read_csv(almost_perfect_file, header = 0)
alpSlp = almostPref['SalePrice']

kg = rmsle(alpSlp, Hs_test_predict)+ 0.04705
print("Kaggle Possible: %s\n" % kg)


In [ ]:
#kaggle_lscv_114180 score: 0.12256
#diff: 0.012745743485

kaggle_file = "c:\\house\\kaggle_lscv_114180.csv"
kaggle_pred = pd.read_csv(kaggle_file, header = 0)
kgSlp = kaggle_pred['SalePrice']

almost_perfect_file = "c:\\house\\almost_perfect.csv"
almostPref = pd.read_csv(almost_perfect_file, header = 0)
alpSlp = almostPref['SalePrice']

kg = rmsle(alpSlp, kgSlp)
print("Kaggle Possible: %s\n" % kg)

In [ ]:
#kaggle_ls_116871 score: 0.12465
#diff: 0.008503387627
    
kaggle_file = "c:\\house\\kaggle_ls_116871.csv"
kaggle_pred = pd.read_csv(kaggle_file, header = 0)
kgSlp = kaggle_pred['SalePrice']

almost_perfect_file = "c:\\house\\almost_perfect.csv"
almostPref = pd.read_csv(almost_perfect_file, header = 0)
alpSlp = almostPref['SalePrice']

kg = rmsle(alpSlp, kgSlp)
print("Kaggle Possible: %s\n" % kg)

In [ ]:
#kaggle_lscv_117731.csv score: 0.12328
#diff: 0.011681495366

kaggle_file = "c:\\house\\kaggle_lscv_117731.csv"
kaggle_pred = pd.read_csv(kaggle_file, header = 0)
kgSlp = kaggle_pred['SalePrice']

almost_perfect_file = "c:\\house\\almost_perfect.csv"
almostPref = pd.read_csv(almost_perfect_file, header = 0)
alpSlp = almostPref['SalePrice']

kg = rmsle(alpSlp, kgSlp)
print("Kaggle Possible: %s\n" % kg)